In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
import glob
import json
from skimage import io,transform
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
train_df=pd.read_json(r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\data/train.json')
#pred_df=pd.read_json(r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\data/test.json')

### split the raw data into train and validation sets

In [3]:
train_df,val_df=train_test_split(train_df,test_size=0.4,random_state=9,shuffle=True,stratify=train_df['is_iceberg'])

In [4]:
#pred_df.head(3)

In [5]:
train_df.head(3)

,band_1,band_2,id,inc_angle,is_iceberg
135,"[-22.52327, -20.36145, -18.631905, -19.001574,...","[-26.236002, -27.819691, -25.408276, -25.14897...",6706ea46,36.1091,1
425,"[-24.873583, -23.380947, -23.155365, -24.60641...","[-24.873583, -24.34704, -25.434231, -26.033535...",32b4a247,45.2814,1
1593,"[-28.554342, -26.37149, -25.17157, -25.171608,...","[-26.055567, -27.394543, -28.55442, -30.389864...",d86deb2b,na,0


In [6]:
def generate_minibatch(inputs=None,batch_size=128,shuffle=True):
    data_cnt=inputs.shape[0]
    if batch_size>data_cnt:
        raise('batch bigger than data nmber')
    if shuffle:
        df=inputs.sample(data_cnt)
    else:
        df=inputs
    start_idx=0
    end_idx=batch_size
    while True:
        if end_idx<start_idx:
            batch_df=df.iloc[start_idx:].append(df.iloc[:end_idx])
        else:
            batch_df=df.iloc[start_idx:end_idx]
        start_idx=(start_idx+batch_size)%data_cnt
        end_idx=(start_idx+batch_size)%data_cnt
        band1=np.array([x for x in batch_df['band_1']])
        band2=np.array([x for x in batch_df['band_2']])
        imgs=np.stack([band1.reshape(-1,75,75),band2.reshape(-1,75,75)],axis=-1)
        imgs=imgs.astype(np.float32)
        labels=batch_df['is_iceberg'].astype(np.int8).values
        yield imgs,labels

In [7]:
train_img,train_label=next(generate_minibatch(inputs=train_df,batch_size=train_df.shape[0]))
test_img,test_label=next(generate_minibatch(inputs=val_df,batch_size=val_df.shape[0]))

#  构建网络

In [8]:
def inference(input_tensor,keep_prob,regularizer):
    with tf.variable_scope('layer1-conv1') as scope:
        conv1_weights=tf.get_variable(name='weight',shape=[3,3,2,64],initializer=tf.random_normal_initializer(stddev=0.01,mean=0,seed=9))
        conv1_biases=tf.get_variable(name='bias',shape=[64],initializer=tf.constant_initializer(0.01))
        conv1=tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
        tf.summary.histogram('layer1-conv1-w',conv1_weights)
        tf.summary.histogram('layer1-conv1-b',conv1_biases)
    with tf.name_scope('layer2-pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer3-conv2'):
        conv2_weights=tf.get_variable(name='weight',shape=[5,5,64,64],initializer=tf.truncated_normal_initializer(stddev=0.01,mean=0.0,seed=9))
        conv2_biases=tf.get_variable(name='bias',shape=64,initializer=tf.constant_initializer(0.01))
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.name_scope('layer4-pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer5-conv3'):
        conv3_weights=tf.get_variable('weight',shape=[3,3,64,32],initializer=tf.truncated_normal_initializer(stddev=0.01,mean=0.0,seed=9))
        conv3_biases=tf.get_variable('bias',shape=[32],initializer=tf.constant_initializer(0.0))
        conv3=tf.nn.conv2d(pool2,conv3_weights,strides=[1,1,1,1],padding='SAME')
        relu3=tf.nn.relu(tf.nn.bias_add(conv3,conv3_biases))
    with tf.name_scope('layer6-pool3'):
        pool3=tf.nn.max_pool(relu3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    
    with tf.variable_scope('layer7-conv4'):
        conv4_weights=tf.get_variable(name='weight',shape=[3,3,32,64],initializer=tf.truncated_normal_initializer(stddev=0.01,mean=0.0,seed=9))
        conv4_biases=tf.get_variable(name='bias',shape=[64],initializer=tf.constant_initializer(0.01))
        conv4=tf.nn.conv2d(pool3,conv4_weights,strides=[1,1,1,1],padding='SAME')
        relu4=tf.nn.relu(tf.nn.bias_add(conv4,conv4_biases))
    with tf.name_scope('layer8-pool4'):
        pool4=tf.nn.max_pool(relu4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        nodes=4*4*64
        flatened=tf.reshape(pool4,[-1,nodes])
    
    with tf.variable_scope('layer9-fc1'):
        fc1_weights=tf.get_variable(name='weight',shape=[nodes,1024],initializer=tf.truncated_normal_initializer(stddev=0.01))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases=tf.get_variable(name='bias',shape=[1024],initializer=tf.constant_initializer(0.01))
        fc1=tf.nn.relu(tf.matmul(flatened,fc1_weights)+fc1_biases)
        fc1=tf.nn.dropout(fc1,keep_prob)
    
    with tf.variable_scope('layer10-fc2'):
        fc2_weights=tf.get_variable(name='weight',shape=[1024,512],initializer=tf.truncated_normal_initializer(stddev=0.01))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases=tf.get_variable(name='bias',shape=[512],initializer=tf.constant_initializer(0.01))
        fc2=tf.nn.relu(tf.matmul(fc1,fc2_weights)+fc2_biases)
        fc2=tf.nn.dropout(fc2,keep_prob)
    
    with tf.variable_scope('layer11-fc3'):
        fc3_weights=tf.get_variable(name='weight',shape=[512,2],initializer=tf.truncated_normal_initializer(stddev=0.01))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc3_weights))
        fc3_biases=tf.get_variable(name='bias',shape=[2],initializer=tf.constant_initializer(0.01))
        logit=tf.matmul(fc2,fc3_weights)+fc3_biases
    return logit

In [9]:
img_h=75
img_w=75
img_c=2
n_steps=400
batch_size=128
model_path=r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\model\cnn_model/cnn.ckpt'

In [10]:
conv_graph=tf.Graph()

In [ ]:
with conv_graph.as_default():
    x=tf.placeholder(shape=[None,img_w,img_h,img_c],dtype=tf.float32,name='input_imgs')
    y=tf.placeholder(shape=[None],dtype=tf.int32,name='input_labels')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    regularizer=tf.contrib.layers.l2_regularizer(0.0001)
    logits=inference(x,keep_prob,regularizer)
    regu_loss=tf.reduce_sum(tf.get_collection('losses'))
    loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y))
    total_loss=loss+regu_loss
    tf.summary.scalar('loss',loss)
    train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(total_loss)
    saver=tf.train.Saver()

# 训练模型

In [ ]:
with tf.Session(graph=conv_graph) as sess:
    summary_merge=tf.summary.merge_all()
    train_writer=tf.summary.FileWriter(r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\model\cnn_info_view/batch128/train/',sess.graph)
    test_writer=tf.summary.FileWriter(r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\model\cnn_info_view/batch128/test/',sess.graph)
    start_time=time.clock()
    sess.run(tf.global_variables_initializer())
    train_loss=0.0
    i=0
    for img,label in generate_minibatch(train_df,batch_size):
        sess.run(train_op,feed_dict={x:img,y:label,keep_prob:0.5})
        #write summary info into logs
        if i%10==0:
            train_infos=sess.run(summary_merge,feed_dict={x:train_img,y:train_label,keep_prob:1.0})
            test_infos=sess.run(summary_merge,feed_dict={x:test_img,y:test_label,keep_prob:1.0})
            train_writer.add_summary(train_infos,i)
            test_writer.add_summary(test_infos,i)
        if i%20==0:
            train_loss=sess.run(loss,feed_dict={x:train_img,y:train_label,keep_prob:1.0})
            test_loss=sess.run(loss,feed_dict={x:test_img,y:test_label,keep_prob:1.0})
            print('train_loss:',train_loss,';  test_loss:',test_loss)
        i+=1
        if i>n_steps:
            break
    end_time=time.clock()
    print('Time:',end_time-start_time)
    saver.save(sess,model_path)

train_loss: 0.692982 ;  test_loss: 0.692975
train_loss: 0.641184 ;  test_loss: 0.637685
train_loss: 0.558514 ;  test_loss: 0.565213
train_loss: 0.545457 ;  test_loss: 0.555825
train_loss: 0.540922 ;  test_loss: 0.550939
train_loss: 0.525789 ;  test_loss: 0.570677
train_loss: 0.51272 ;  test_loss: 0.522989
train_loss: 0.847079 ;  test_loss: 0.851597
train_loss: 0.493065 ;  test_loss: 0.529151
train_loss: 0.405668 ;  test_loss: 0.463972
train_loss: 0.334774 ;  test_loss: 0.40449
train_loss: 0.325349 ;  test_loss: 0.388571
train_loss: 0.330824 ;  test_loss: 0.388075


# Predict

In [2]:
model_dir=r"F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\model\cnn_model/"
pred_data_path=r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\data/test.json'

In [3]:
def prepare_data(data_path,batch_size=100):
    with open(data_path) as file:
        lines=file.readlines()
    pred_df=pd.DataFrame(json.loads(lines[0]))
    # 将数据转换成模型需要的格式
    pred_band1=np.array([x for x in pred_df['band_1']])
    pred_band2=np.array([x for x in pred_df['band_2']])
    pred_imgs=np.stack([pred_band1.reshape(-1,75,75),pred_band2.reshape(-1,75,75)],axis=-1)
    pred_imgs=pred_imgs.astype(np.float32)
    img_id=pred_df[['id']]
    img_cnt=pred_imgs.shape[0]
    i=0
    while i<img_cnt:
        end_idx=min(i+batch_size,img_cnt)
        yield pred_imgs[i:end_idx],img_id.iloc[i:end_idx]
        i=i+batch_size

In [4]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(model_dir+"cnn.ckpt.meta")
    saver.restore(sess, model_dir+"cnn.ckpt")
    graph=tf.get_default_graph()
    x=graph.get_tensor_by_name("input_imgs:0")
    keep_prob=graph.get_tensor_by_name("keep_prob:0")
    logits=graph.get_tensor_by_name('layer11-fc3/add:0')
    probs=tf.nn.softmax(logits)
    pred_probs=[]
    k=0
    batch_test=200
    for pred_imgs,img_id in prepare_data(pred_data_path,batch_size=batch_test):
        result=sess.run(probs,feed_dict={x:pred_imgs,keep_prob:1.0})
        pred_prob=result[:,1]
        img_id['is_iceberg']=pred_prob
        pred_probs.append(img_id)
        print('predict:{0}x{1}'.format(k,batch_test))
        k+=1

INFO:tensorflow:Restoring parameters from F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\model\cnn_model/cnn.ckpt


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


predict:0x200
predict:1x200
predict:2x200
predict:3x200
predict:4x200
predict:5x200
predict:6x200
predict:7x200
predict:8x200
predict:9x200
predict:10x200
predict:11x200
predict:12x200
predict:13x200
predict:14x200
predict:15x200
predict:16x200
predict:17x200
predict:18x200
predict:19x200
predict:20x200
predict:21x200
predict:22x200
predict:23x200
predict:24x200
predict:25x200
predict:26x200
predict:27x200
predict:28x200
predict:29x200
predict:30x200
predict:31x200
predict:32x200
predict:33x200
predict:34x200
predict:35x200
predict:36x200
predict:37x200
predict:38x200
predict:39x200
predict:40x200
predict:41x200
predict:42x200


In [7]:
pred_result=pred_probs[0]
for i in range(1,len(pred_probs)):
    pred_result=pred_result.append(pred_probs[i])

In [10]:
pred_result.head()

,id,is_iceberg
0,5941774d,0.000836
1,4023181e,0.742121
2,b20200e4,1.000000
3,e7f018bb,0.960401
4,4371c8c3,0.770937


In [9]:
pred_result.to_csv(r'F:\study\competitions\Iceberg-Classifier\Iceberg Classifier Challenge\20171125_1CNN_pred1.csv',index=False)